In [1]:
from bardapi import Bard
import base64
import json
import os

In [2]:
class ImageAnotator:

    def __init__(self) -> None:
        self.bard = Bard(token=os.environ.get('BARD_TOKEN'))
        self.prompt='''You are an assessment expert responsible for annotate images that will be used on a model's training. Your task is to create an annotation for a creature on a png-image according to the following requirements:


    1. Your answer must be in a valid JSON format with at least one field called “command”, so your fully answer must be between braces;
    2. The “command” field must contain a single paragraph with the annotation;
    3. The annotation must be in a command format, it will be used as the input for the other model;
4. Do not give information that refer to an image, like positioning or background;

Desired response format example:” { “Command”: “Create a creature with the body of a toy. The body should be made of plastic and have a toy-like appearance. The creature should have a simple, child-like design. The creature should be able to move its limbs and head. The creature should have a friendly expression.”}”


Annotate the following image:'''

    def __get_command(self, answer: str) -> str:
        start_json = answer.find('{')
        end_jason = answer.rfind('}')
        json_str = answer[start_json:end_jason+1]

        try:
            file = json.loads(json_str.lower())
            return file['command']
        except:
            return None
    
    def annot_image(self, image_path) -> str:
        
        with open(image_path, "rb") as image_file:
            bard_answer = self.bard.ask_about_image(self.prompt, image_file.read())

        return self.__get_command(bard_answer['content'])


In [3]:
def save_annotation(image_path, annotation):
    # create a folder to save the annotations
    sample_name = image_path.split('/')[-1].split('.')[0]

    if not os.path.exists(f'./data/{sample_name}'):
        os.mkdir(f'./data/{sample_name}')

    with open(f'./data/{sample_name}/annotation.txt', 'w') as f:
        f.write(annotation)

    #move image to the folder
    with open(image_path, 'rb') as f:
        image_string = base64.b64encode(f.read()).decode('utf-8')

    with open(f'./data/{sample_name}/image.png', 'wb') as f:
        f.write(image_string)
    
    os.remove(image_path)

In [4]:
from tqdm import tqdm


In [5]:
while True:
    try:
        annotator = ImageAnotator()

        for image in tqdm(os.listdir('images')):
            annotation = None
            tries = 0

            image_path = f'./images/{image}'

            while annotation is None and tries < 3:
                tries += 1
                annotation = annotator.annot_image(image_path)

            tries = 0
            if annotation is None:
                os.rename(image_path, f'./annot_fail/{image}')
                continue
            save_annotation(image_path, annotation)
    except:
        pass

  0%|          | 0/38280 [00:00<?, ?it/s]